In [2]:
# get yelp data
import wget
wget.download('https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz')

100% [......................................................................] 166373201 / 166373201

'yelp_review_polarity_csv.tgz'

In [30]:
# import key functions and modules from fastai
from fastai import * 
from fastai.text import * 
from fastai.core import *
import pandas as pd

OSError: [WinError 1455] 이 작업을 완료하기 위한 페이징 파일이 너무 작습니다. Error loading "C:\Users\2joon\anaconda3\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [20]:
# set path to files
import os
os.chdir("C:/Users/2joon/John Oh/yelp_review_polarity_csv")
bs=48

In [24]:
# load training and view data
train_csv = 'train.csv'
train = pd.read_csv(train_csv, header=None) 
train.head()

,0,1
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [ ]:
train.iloc[0][1]

In [25]:
train[0].unique()

array([1, 2], dtype=int64)

In [ ]:
# load and view test/validation data

In [26]:
valid_csv = 'test.csv'
valid = pd.read_csv(valid_csv, header=None) 
valid.head()

,0,1
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


In [27]:
valid[0].unique()

array([2, 1], dtype=int64)

In [31]:
data_lm = TextLMDataBunch.from_csv(path, 'test.csv')
data_clas = TextClasDataBunch.from_csv(path, 'test.csv', vocab=data_lm.train_ds.vocab, bs=bs)

NameError: name 'TextLMDataBunch' is not defined

In [ ]:
data_lm.show_batch()

In [ ]:
data_clas.show_batch()

In [ ]:
data_clas.vocab.itos[:10]

In [ ]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/' 
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth') 
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [ ]:
# initialize lang model using pre-trained lang models

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=['lstm_wt103', 'itos_wt103'], drop_mult=0.5)

In [ ]:
learn.predict('This was such a great ', 50, temperature=1.1, min_p=0.001)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(7, 1e-3,moms=(0.8,0.7))

In [ ]:
# save fine-tuned model

In [ ]:
learn.save_encoder('ft_enc')

In [ ]:
learn.predict('It was nice ', 50, temperature=1.1, min_p=0.001)

In [ ]:
learn.predict('We are never going ', 25, temperature=1.1, min_p=0.001)

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.load_encoder('ft_enc')
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.save('second')

In [ ]:
learn.load('second')

In [ ]:
s = '''
The Big Boy burger remains a favorite, but they have lost me on there breakfast.  
This is especially true for this store.  Ugh.
'''

In [ ]:
prediction = learn.predict(s)

In [ ]:
prediction

In [ ]:
learn.data.classes